## Heart Disease - EDA,FE and ML Prediction

The data includes 303 patient level features including if they have heart disease at the end or not. Features are like;

- Age: Obvious one...
- Sex:
    - 0: Female
    - 1: Male
- Chest Pain Type: 
    - 0: Typical Angina
    - 1: Atypical Angina
    - 2: Non-Anginal Pain
    - 3: Asymptomatic
- Resting Blood Pressure: Person's resting blood pressure.
- Cholesterol: Serum Cholesterol in mg/dl  
- Fasting Blood Sugar:
    - 0:Less Than 120mg/ml
    - 1: Greater Than 120mg/ml
- Resting Electrocardiographic Measurement:
    - 0: Normal
    - 1: ST-T Wave Abnormality
    - 2: Left Ventricular Hypertrophy
- Max Heart Rate Achieved: Maximum Heart Rate Achieved
- Exercise Induced Angina:
    - 1: Yes
    - 0: No
- ST Depression: ST depression induced by exercise relative to rest.
- Slope: Slope of the peak exercise ST segment:
    - 0: Upsloping
    - 1: Flat
    - 2: Downsloping
- Thalassemia: A blood disorder called 'Thalassemia':
    - 0: Normal
    - 1: Fixed Defect
    - 2: Reversable Defect
- Number of Major Vessels: Number of major vessels colored by fluoroscopy.

https://github.com/HimanshuKGP007/HeartDiseaseAnalysis

In [ ]:
pip install autoviz

In [ ]:
# Loading packages.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from autoviz.AutoViz_Class import AutoViz_Class
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from pandas_profiling import ProfileReport
import math
import random
import os
import time
# Setup

# common:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import folium

# preprocessing
import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, learning_curve, ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, confusion_matrix, explained_variance_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel, SelectKBest, RFE, chi2
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC


# models
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

# NN models
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier


# set some display options:
sns.set(style="whitegrid")
pd.set_option("display.max_columns", 36)

import warnings
warnings.filterwarnings('ignore')
from numpy import interp

# disabling warnings.

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
# Styling...
cust_palt = [
    '#111d5e', '#c70039', '#f37121', '#ffbd69', '#ffc93c'
]

plt.style.use('ggplot')

In [ ]:
# Reading csv file:

train = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')

In [ ]:
# Taking random samples from data.

train.head(5)

In [ ]:
train.info()

In [ ]:
train.columns

In [ ]:
# Renaming columns.
train.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar',
                 'rest_ecg', 'max_heart_rate_achieved','exercise_induced_angina', 'st_depression', 'st_slope',
                 'num_major_vessels', 'thalassemia', 'condition']

In [ ]:
# Renaming cateorical data for easier understanding:

train['sex'] = train['sex'].map({0:'female',1:'male'})
train['chest_pain_type'] = train['chest_pain_type'].map({3:'asymptomatic', 1:'atypical_angina', 2:'non_anginal_pain', 0:'typical_angina'})
train['fasting_blood_sugar'] = train['fasting_blood_sugar'].map({0:'less_than_120mg/ml',1:'greater_than_120mg/ml'})
train['rest_ecg'] = train['rest_ecg'].map({0:'normal',1:'ST-T_wave_abnormality',2:'left_ventricular_hypertrophy'})
train['exercise_induced_angina'] = train['exercise_induced_angina'].map({0:'no',1:'yes'})
train['st_slope'] = train['st_slope'].map({0:'upsloping',1:'flat',2:'downsloping'})
train['thalassemia'] = train['thalassemia'].map({1:'fixed_defect',0:'normal',2:'reversable_defect'})
train['condition'] = train['condition'].map({0:'no_disease', 1:'has_disease'})


## Univariate Analysis

#### For this part we going to inspect how's the data distribution is and what patterns we can inspect.

In [ ]:
# Masks for easier selection in future:
# Very Important
categorical = [i for i in train.loc[:,train.nunique()<=10]]
continuous = [i for i in train.loc[:,train.nunique()>=10]]

In [ ]:
[i for i in train.loc[:,train.nunique()<=10]]

In [ ]:
[i for i in train.loc[:,train.nunique()>=10]]

In [ ]:
total = float(len(train[categorical]))

In [ ]:
def ctg_dist(df, cols, hue,rows, columns):
    
    '''A function for displaying cateorical distribution'''
    
    fig, axes = plt.subplots(rows, columns, figsize=(16, 12))
    axes = axes.flatten()

    for i, j in zip(df[cols].columns, axes):
        sns.countplot(x=i,
                    data=df,
                    ax=j,
                    order=df[i].value_counts().index)
        j.tick_params(labelrotation=10)
        j.set_title(f'{str(i).capitalize()} Distribution')
        plt.tight_layout()
        total = float(len(df[i]))        
        for p in j.patches:
            height = p.get_height()
            j.text(p.get_x() + p.get_width() / 2.,
                    height + 2,
                    '{:1.2f}%'.format((height / total) * 100),
                    ha='center')

In [ ]:
total

## Categorical Data

### Here we can do these observations:
- Males on the dataset is more than double of the female observations.
- Most common ches pain type is 'Asymptomatic' ones which is almost 50% of the data
- 85% of the patients has no high levels of fastin blood sugar.
- Resing electrocardiographic observations are evenly distributed between normal and left ventricular hypertrophy with ST-T minority
- 67% of the patients had no exercise induced angina
- Peak exercise slope seems mainly divided between upsloping and flat.

In [ ]:
# Display categorical columns:

ctg_dist(train, categorical,hue=None, rows=3,columns=3)

## Numerical Data

### Most of the continuous variables somewhat close to gaussian distribution with small skews left or right except for oldpeak. Again there are some outliers espacially a strong one in Cholesterol worth to take a look later.

In [ ]:
# Displaying numeric distribution:

fig = plt.figure(constrained_layout=True, figsize=(16, 12))


grid = gridspec.GridSpec(ncols=6, nrows=3, figure=fig)

ax1 = fig.add_subplot(grid[0, :2])

ax1.set_title('Trestbps Distribution')

sns.distplot(train[continuous[1]],
                 hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8})

ax15 = fig.add_subplot(grid[0, 2:3])

ax15.set_title('Trestbps')

sns.boxplot(train[continuous[1]], orient='v')

ax2 = fig.add_subplot(grid[0, 3:5])

ax2.set_title('Chol Distribution')

sns.distplot(train[continuous[2]],
                 hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8})

ax25 = fig.add_subplot(grid[0, 5:])

ax25.set_title('Chol')

sns.boxplot(train[continuous[2]], orient='v')

ax3 = fig.add_subplot(grid[1, :2])

ax3.set_title('Thalach Distribution')

sns.distplot(train[continuous[3]],
                 hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8})

ax35 = fig.add_subplot(grid[1, 2])

ax35.set_title('Thalach')

sns.boxplot(train[continuous[3]], orient='v')

ax4 = fig.add_subplot(grid[1, 3:5])

ax4.set_title('Oldpeak Distribution')

sns.distplot(train[continuous[4]],
                 hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8})

ax45 = fig.add_subplot(grid[1, 5:])

ax45.set_title('Oldpeak')

sns.boxplot(train[continuous[4]], orient='v')

ax5 = fig.add_subplot(grid[2, :4])

ax5.set_title('Age Distribution')

sns.distplot(train[continuous[0]],
                 hist_kws={
                 'rwidth': 0.95,
                 'edgecolor': 'black',
                 'alpha': 0.8})

ax55 = fig.add_subplot(grid[2, 4:])

ax55.set_title('Age')

sns.boxplot(train[continuous[0]], orient='h')

plt.show()


In [ ]:
def ctg_dist1(df, cols,rows, columns):
    
    '''A function for displaying cateorical distribution'''
    
    fig, axes = plt.subplots(rows, columns, figsize=(20, 15))
    axes = axes.flatten()

    for i, j in zip(df[cols].columns, axes):
        sns.countplot(x=i,
                    data=df,
                    ax=j,
                    order=df[i].value_counts().index,
                    hue='condition')
        j.tick_params(labelrotation=10)
        j.set_title(f'{str(i).capitalize()} Distribution')
        total = float(len(df[i]))        
        for p in j.patches:
            height = p.get_height()
            j.text(p.get_x() + p.get_width() / 2.,
                    height + 2,
                    '{:1.2f}%'.format((height / total) * 100),
                    ha='center')
        plt.tight_layout()

# Bivariate Analysis
## Categorical Data vs Target

### Here we can do these observations:

- Males are much more likely for heart diseases.
- Chest pain type is very subjective and has no direct relation on the outcome, asymptomatic chest pains having highest disease outcome.
- Blood sugar has no direct effect on the disease.
- Rest ECG results showing no direct results but having normal ECG is pretty good sign. Even though it's pretty rare in the data, if you ST-T wave abnormality you are 3 times more likely to have heart disease.
- Having exercise induced angina is pretty strong indicator for heart disease, patients are almost 3 times more likely to have disease if they have exercise induced angina. Meanwhile it's less than half for not having it.
- Patients who had flat slope distribution are more likely to have disease.
- Number of major vessels observed seems on similar levels for patients who have disease but 0 observations is good sign for not having disease.
- Having defected thalium test results is pretty strong indicator for heart disease.

In [ ]:
ctg_dist1(train,categorical,4,2)

## Numerical Data vs Target

### Here we can do these observations:

- Having higher resting blood pressure shows you are little bit more likely to have heart disease.
- Again same for Cholesterol, it's not strong indicator but patients are little bit more likely to have disease with high cholesterol. There's is also one outlier there with no disease, pretty interesting.
- I find max heart rate distribution a bit interesting, expecting the other way around but it might be due to testing conditions and if you have normal results on ECG while exercising instructors might be increasing your excercise density?
- It's pretty clear that heart disease likelihood increases with ST depression levels...
- Lastly older patients are more likely to have heart disease.

In [ ]:
# Displaying numeric distribution vs condition:

fig = plt.figure(constrained_layout=True, figsize=(16, 12))


grid = gridspec.GridSpec(ncols=4, nrows=3, figure=fig)

ax1 = fig.add_subplot(grid[0, :2])

ax1.set_title('resting_blood_pressure Distribution')

sns.boxplot(x='condition',
                    y='resting_blood_pressure',
                    data=train,
                    palette=cust_palt[2:],
                    ax=ax1)
sns.swarmplot(x='condition',
                    y='resting_blood_pressure',
                    data=train,
                    palette=cust_palt[:2],
                    ax=ax1)

ax2 = fig.add_subplot(grid[0, 2:])

ax2.set_title('cholesterol Distribution')

sns.boxplot(x='condition',
                    y='cholesterol',
                    data=train,
                    palette=cust_palt[2:],
                    ax=ax2)
sns.swarmplot(x='condition',
                    y='cholesterol',
                    data=train,
                    palette=cust_palt[:2],
                    ax=ax2)

ax3 = fig.add_subplot(grid[1, :2])

ax3.set_title('max_heart_rate_achieved Distribution')

sns.boxplot(x='condition',
                    y='max_heart_rate_achieved',
                    data=train,
                    palette=cust_palt[2:],
                    ax=ax3)
sns.swarmplot(x='condition',
                    y='max_heart_rate_achieved',
                    data=train,
                    palette=cust_palt[:2],
                    ax=ax3)

ax4 = fig.add_subplot(grid[1, 2:])

ax4.set_title('st_depression Distribution')

sns.boxplot(x='condition',
                    y='st_depression',
                    data=train,
                    palette=cust_palt[2:],
                    ax=ax4)
sns.swarmplot(x='condition',
                    y='st_depression',
                    data=train,
                    palette=cust_palt[:2],
                    ax=ax4)

ax5 = fig.add_subplot(grid[2, :])

ax5.set_title('age Distribution')

sns.boxplot(x='condition',
                    y='age',
                    data=train,
                    palette=cust_palt[2:],
                    ax=ax5)
sns.swarmplot(x='condition',
                    y='age',
                    data=train,
                    palette=cust_palt[:2],
                    ax=ax5)
plt.show()

# Multivariate Analysis

In [ ]:
# Numeric data vs each other and condition:

plt.figure(figsize=(16, 10))
sns.pairplot(train[['resting_blood_pressure','cholesterol','max_heart_rate_achieved','st_depression','age', 'condition']], hue='condition',
           markers=['o','D'], plot_kws=dict(s=25, alpha=0.75, ci=None)
            )

plt.show()

In [ ]:
# 3D scatterplot of numeric data:

fig = px.scatter_3d(train, x='cholesterol', y='max_heart_rate_achieved', z='age', size='st_depression',
              color='condition', opacity=0.8)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## Correlations

In [ ]:
corr_train = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')
full_data = train

In [ ]:
corr_train.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar',
                 'rest_ecg', 'max_heart_rate_achieved','exercise_induced_angina', 'st_depression', 'st_slope',
                 'num_major_vessels', 'thalassemia', 'condition']

In [ ]:
sns.set(font_scale=1.3)
corr_categorical=corr_train.corr(method='spearman')
mask_categorical = np.triu(np.ones_like(corr_categorical, dtype=np.bool))
plt.figure(figsize=(20, 12))
sns.heatmap(corr_categorical, annot=True, fmt=".2f", cmap='BrBG', vmin=-1, vmax=1, center= 0,
            square=True, linewidths=2, cbar_kws={"shrink": .5}).set(ylim=(14, 0))
plt.title("Correlation Matrix Spearman Method- Categorical Data ",size=15, weight='bold')


In [ ]:
sns.set(font_scale=1.3)
corr_categorical=corr_train.corr(method='pearson')
mask_categorical = np.triu(np.ones_like(corr_categorical, dtype=np.bool))
plt.figure(figsize=(20, 12))
sns.heatmap(corr_categorical, annot=True, fmt=".2f", cmap='BrBG', vmin=-1, vmax=1, center= 0,
            square=True, linewidths=2, cbar_kws={"shrink": .5}).set(ylim=(14, 0))
plt.title("Correlation Matrix Spearman Method- Categorical Data ",size=15, weight='bold')


In [ ]:
correlations = corr_train.corrwith(corr_train['condition']).iloc[:-1].to_frame()
correlations[0] = correlations[0].abs()
correlations.sort_values(by=0, inplace=True, ascending=False)
labels = correlations.index
ax = sns.boxplot(x=0,y=labels, data=correlations, palette = 'pink')
ax.set_yticklabels(labels)
ax.set(xlabel='Correlation', ylabel='Parameter')

## Feature Engineering

In [ ]:
data = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')

In [ ]:
def fe_creation(df):
    df['age2'] = df['age']//10
    df['trestbps2'] = df['trestbps']//10
    df['chol2'] = df['chol']//40
    df['thalach2'] = df['thalach']//40
    df['oldpeak2'] = df['oldpeak']//0.4
    for i in ['sex', 'age2', 'fbs', 'restecg', 'exang','thal', ]:
        for j in ['cp','trestbps2', 'chol2', 'thalach2', 'oldpeak2', 'slope', 'ca']:
            df[i + "_" + j] = df[i].astype('str') + "_" + df[j].astype('str')
    return df

data = fe_creation(data)

In [ ]:
data.columns

In [ ]:
data.head(3)

In [ ]:
categorical_columns = {'sex_cp', 'sex_trestbps2',
       'sex_chol2', 'sex_thalach2', 'sex_oldpeak2', 'sex_slope', 'sex_ca',
       'age2_cp', 'age2_trestbps2', 'age2_chol2', 'age2_thalach2',
       'age2_oldpeak2', 'age2_slope', 'age2_ca', 'fbs_cp', 'fbs_trestbps2',
       'fbs_chol2', 'fbs_thalach2', 'fbs_oldpeak2', 'fbs_slope', 'fbs_ca',
       'restecg_cp', 'restecg_trestbps2', 'restecg_chol2', 'restecg_thalach2',
       'restecg_oldpeak2', 'restecg_slope', 'restecg_ca', 'exang_cp',
       'exang_trestbps2', 'exang_chol2', 'exang_thalach2', 'exang_oldpeak2',
       'exang_slope', 'exang_ca', 'thal_cp', 'thal_trestbps2', 'thal_chol2',
       'thal_thalach2', 'thal_oldpeak2', 'thal_slope', 'thal_ca'}

In [ ]:
#Label Encoding the Categorical Columns
for col in categorical_columns:
      le = LabelEncoder()
      le.fit(list(data[col].astype(str).values))
      data[col] = le.transform(list(data[col].astype(str).values))

In [ ]:
data.head(3)

### Feature Selection

In [ ]:
train = data.copy()
target = train.pop('condition')
train.head(2)

In [ ]:
num_features_opt = 25   # the number of features that we need to choose as a result
num_features_max = 35   # the somewhat excessive number of features, which we will choose at each stage
features_best = []

In [ ]:
#FS by the SelectFromModel with LinearSVC 
lsvc = LinearSVC(C=0.1, penalty="l1", dual=False).fit(train, target)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(train)
X_selected_df = pd.DataFrame(X_new, columns=[train.columns[i] for i in range(len(train.columns)) if model.get_support()[i]])
features_best.append(X_selected_df.columns.tolist())

In [ ]:
#FS by the SelectFromModel with Lasso
lasso = LassoCV(cv=3).fit(train, target)
model = SelectFromModel(lasso, prefit=True)
X_new = model.transform(train)
X_selected_df = pd.DataFrame(X_new, columns=[train.columns[i] for i in range(len(train.columns)) if model.get_support()[i]])
features_best.append(X_selected_df.columns.tolist())

In [ ]:
#FS by the SelectKBest with Chi-2
bestfeatures = SelectKBest(score_func=chi2, k='all')
fit = bestfeatures.fit(train, target)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(train.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Feature','Score']
featureScores = featureScores.sort_values(by='Score', ascending=False)
features_best.append(featureScores['Feature'].to_list())

In [ ]:
#FS by the Recursive Feature Elimination (RFE) with Logistic Regression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_features_max, step=10, verbose=5)
rfe_selector.fit(train, target)
rfe_support = rfe_selector.get_support()
rfe_feature = train.loc[:,rfe_support].columns.tolist()
features_best.append(rfe_feature)

In [ ]:
#FS by the VarianceThreshold
selector = VarianceThreshold(threshold=10)
np.shape(selector.fit_transform(data))
features_best.append(list(np.array(data.columns)[selector.get_support(indices=False)]))

In [ ]:
features_best

In [ ]:
# Most common items in all lists of optimal features
main_cols = []
main_cols_opt = {feature_name : 0 for feature_name in data.columns.tolist()}
for i in range(len(features_best)):
  for feature_name in features_best[i]:
        main_cols_opt[feature_name] += 1
df_main_cols_opt = pd.DataFrame.from_dict(main_cols_opt, orient='index', columns=['Num'])
df_main_cols_opt.sort_values(by=['Num'], ascending=False)

In [ ]:
main_cols = df_main_cols_opt.nlargest(num_features_opt, 'Num').index.tolist()

In [ ]:
len(main_cols)

## ML Prediction

In [ ]:
X = train[main_cols]
y = target

In [ ]:
# define models to test:
base_models = [("DT_model", DecisionTreeClassifier(random_state=42)),
               ("RF_model", RandomForestClassifier(random_state=42,n_jobs=-1)),
               ("LR_model", LogisticRegression(random_state=42,n_jobs=-1)),
               ("XGB_model", XGBClassifier(random_state=42, n_jobs=-1)),
               ("SVM_model",LinearSVC(random_state=42)),
               ("NB_model", GaussianNB())]
# split data into 'kfolds' parts for cross validation,
# use shuffle to ensure random distribution of data:
kfolds = 4 # 4 = 75% train, 25% validation
split = KFold(n_splits=kfolds, shuffle=True, random_state=42)

for name, model in base_models:   
    # get cross validation score for each model:
    cv_results = cross_val_score(model, 
                                 X, y, 
                                 cv=split,
                                 scoring="accuracy",
                                 n_jobs=-1)
    # output:
    min_score = round(min(cv_results), 4)
    max_score = round(max(cv_results), 4)
    mean_score = round(np.mean(cv_results), 4)
    std_dev = round(np.std(cv_results), 4)
    print(f"{name} cross validation accuarcy score: {mean_score} +/- {std_dev} (std) min: {min_score}, max: {max_score}, ")

Logistic Regression Model gives the best accuracy at 83.52% 

In [ ]:
#HyperParameterTuning for LR Model 
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01, 101, 99, 102, 98]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

As we can see, after Hyper Parameter tuning of LogisticRegression we get accuracy of 84.26%